In [ ]:
#open files
mode1 = 'classification system 1'
mode2 = 'classificaiton system 2'
filename = 'dataset file*.xlsx'
inputSheet = 'input sheet name'  
openSheet = 'aw_'+mode1+'_cn'
kow2v_model ='ko_w2v_defect-0710-v2.bin'  # fine-tuned word2vec file 

import re
import gensim
import pandas as pd
import numpy as np
import scipy.sparse as ss
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from scipy import spatial
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
%matplotlib inline

#feature vec = 200
kovec = Word2Vec.load('C:/Users/ImKH/2021DefectClassify/ko-w2v/'+kow2v_model)

#Preprocessing 
import MeCab
import nltk
from tqdm import tqdm
from konlpy.tag import Mecab 
from konlpy.tag import *

mc = Mecab(dicpath='he path of the MeCab-ko dictionary.') # The path of the MeCab-ko dictionary.
m = MeCab.Tagger("-O wakati")

class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    
    def __call__(self, sent):
        postags=['NNP', 'NNG', 'VV', 'VA','SL', 'VV+ETN'] 
        pos = self.tagger.pos(sent)
        pos = [word for (word, pos) in mc.pos(sent, flatten=True) if pos in postags and len(word)>1]
 
        return pos

my_tokenizer = MyTokenizer(Mecab(dicpath='C:\mecab\mecab-ko-dic'))

# Defect Data load 
dldefect_df = pd.read_excel("C:/Users/ImKH/2021DefectClassify/data/"+filename, sheet_name=inputSheet)
rawComplaints = dldefect_df.apply(lambda row: " ".join(re.sub("[^a-zA-Z가-힣]+"," ", row.complaint).split()), 1).to_list()
dldefect_df

In [ ]:
#Extract random sample  
# UDF of random sampling 
def sampling_func(data, sample_pct):
    np.random.seed(123)
    N = len(data)
    if sample_pct <1:
        sample_n = int(len(data)*sample_pct) # integer
    else:
        sample_n = sample_pct
    sample = data.take(np.random.permutation(N)[:sample_n])
    return sample

#random sampling  
mode1_DF = dldefect_df.groupby(mode1, group_keys=False).apply(sampling_func, sample_pct=1000)
mode1_DF = mode1_DF.sample(frac=1).reset_index(drop=True)
mode1_raw = mode1_DF.apply(lambda row: " ".join(re.sub("[^a-zA-Z가-힣]+"," ", row.complaint).split()), 1).to_list()

mode2_DF = dldefect_df.groupby(mode2, group_keys=False).apply(sampling_func, sample_pct=1000)
mode2_DF = mode2_DF.sample(frac=1).reset_index(drop=True)
mode2_raw = mode2_DF.apply(lambda row: " ".join(re.sub("[^a-zA-Z가-힣]+"," ", row.complaint).split()), 1).to_list()



In [25]:
def getSentenceMeanEmbeddings (sentenceCorpus):
    sent_embdding_list =[]
    for line in sentenceCorpus:
        sen2vec = None
        count=0
        for word in my_tokenizer(line):
#             print(word)
            if word in kovec.wv.vocab:
                count +=1
                if sen2vec is None:
                    sen2vec = kovec[word]
                else: 
                    sen2vec = sen2vec+ kovec[word]
        if sen2vec is not None:
            sen2vec = sen2vec/count
            sent_embdding_list.append(sen2vec)
            
    return sent_embdding_list

In [ ]:
s1_emblist = getSentenceMeanEmbeddings (mode1_raw)
s2_emblist = getSentenceMeanEmbeddings (mode2_raw)

print(len(s1_emblist), len(s2_emblist))

In [ ]:
sent_embdding_list = getSentenceMeanEmbeddings (mode1_raw)
print(len(mode1_raw), len(sent_embdding_list))

In [42]:
from numpy import dot
from numpy.linalg import norm 


def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

cos_score=[]
datalen = min(len(s1_emblist), len(s2_emblist))
for i in tqdm(range(0, datalen)):
    cos_score.append(round(cos_sim(s1_emblist[i], s2_emblist[i]), 2))

100%|█████████████████████████████████████████████████████████████████████████| 13142/13142 [00:00<00:00, 83911.62it/s]


In [43]:
stsDF = pd.DataFrame({'s1': rawComplaints[:datalen], 's2': rawComplaints2[:datalen], 'cosSim': cos_score})
stsDF

,s1,s2,cosSim
0,욕실 바닥타일 들뜸 바닥타일 오타공 곳 월 일 욕실 벽타일 바닥타일 단차,침실 PL창호 수직수평불량 해피콜금지 분합창 하부수평불량 문이휨 작업자 업체에보고한...,0.47
1,거실 월패드 작동불량 완료 월패드 지하주차장 통화음 미작동 팩스전송,현관 환기커버 고정불량 현관과 창고내 배기구 흡입이 안된다함 향초로 확인했다함,0.61
2,욕실 바닥타일 단차 부부욕실 바닥 타일 침하 어제 cs담당자 확인한 사항임 센터장 ...,욕실 욕조 고정불량 윤상권완료 욕조사용시 찌지직 소리가 남,0.47
3,침실 벽도배 결로 선진티앤알 침 pl창 우하부 결로 보수후 선진완료 후속도배,거실 벽도배 들뜸 거실 벽지 들뜸과 아래쪽 하단벽지 찢김,0.49
4,팬트리 스위치 작동불량 스위치불량 끌때 잘안꺼지며 세게 눌러야꺼짐,복도 스프링클러 누수 현관에서 들어오는 초입구부분에서 천정 누수가 하루종일 있었다고...,0.58
...,...,...,...
13137,창고 조명기구 점등불량 창고 불 안꺼짐,주방발코니 목문 찢김 긁힘 안쪽문틀 칼로 그은듯 교체요망,0.36
13138,침실 목문 타카불량 문짝 오염 문틀 핀자국 다수,주방발코니 목문 개폐불량 열림 잘되지 않음 여 통화시 현재 개폐는되나 보수이후 바람...,0.56
13139,거실 천정도배 이음불량 거실 천정 도배들뜸 재발생 직영방문 거실 천장 우면 날개벽 ...,주방 식당 주방상판 크랙 쿡탑옆 작은실금 as보수흔적 있음,0.41
13140,거실 마루판 코킹불량 아트월 각실코킹 벌어짐 세대 전체적으로 벌어진곳 많음 담당자 ...,침실 벽체골조 파손 외벽 난간대 우측 콘크리트 깨짐,0.40


In [44]:
stsDF.to_excel('./data/sbert_trainset.xlsx')

In [2]:
staDF = pd.read_excel('./data/sbert_trainset.xlsx')
staDF

,Unnamed: 0,s1,s2,cosSim
0,0,욕실 바닥타일 들뜸 바닥타일 오타공 곳 월 일 욕실 벽타일 바닥타일 단차,침실 PL창호 수직수평불량 해피콜금지 분합창 하부수평불량 문이휨 작업자 업체에보고한...,0.47
1,1,거실 월패드 작동불량 완료 월패드 지하주차장 통화음 미작동 팩스전송,현관 환기커버 고정불량 현관과 창고내 배기구 흡입이 안된다함 향초로 확인했다함,0.61
2,2,욕실 바닥타일 단차 부부욕실 바닥 타일 침하 어제 cs담당자 확인한 사항임 센터장 ...,욕실 욕조 고정불량 윤상권완료 욕조사용시 찌지직 소리가 남,0.47
3,3,침실 벽도배 결로 선진티앤알 침 pl창 우하부 결로 보수후 선진완료 후속도배,거실 벽도배 들뜸 거실 벽지 들뜸과 아래쪽 하단벽지 찢김,0.49
4,4,팬트리 스위치 작동불량 스위치불량 끌때 잘안꺼지며 세게 눌러야꺼짐,복도 스프링클러 누수 현관에서 들어오는 초입구부분에서 천정 누수가 하루종일 있었다고...,0.58
...,...,...,...,...
13137,13137,창고 조명기구 점등불량 창고 불 안꺼짐,주방발코니 목문 찢김 긁힘 안쪽문틀 칼로 그은듯 교체요망,0.36
13138,13138,침실 목문 타카불량 문짝 오염 문틀 핀자국 다수,주방발코니 목문 개폐불량 열림 잘되지 않음 여 통화시 현재 개폐는되나 보수이후 바람...,0.56
13139,13139,거실 천정도배 이음불량 거실 천정 도배들뜸 재발생 직영방문 거실 천장 우면 날개벽 ...,주방 식당 주방상판 크랙 쿡탑옆 작은실금 as보수흔적 있음,0.41
13140,13140,거실 마루판 코킹불량 아트월 각실코킹 벌어짐 세대 전체적으로 벌어진곳 많음 담당자 ...,침실 벽체골조 파손 외벽 난간대 우측 콘크리트 깨짐,0.40


In [3]:
def make_sts_input_example(openfile):
    ''' 
    Transform to InputExample
    ''' 
    total = openfile.shape[0]

    train_num = round(total*0.8)
    val_num = round(total*0.15)
    test_num = total-train_num-val_num

    train_examples = []
    print('train range:',train_num)
    for i in range(0, train_num):
        sentence1 = openfile.iloc[i]['s1']
        sentence2 = openfile.iloc[i]['s2']
        score = openfile.iloc[i]['cosSim']
        train_examples .append({i:[sentence1, sentence2, np.float32(score)]})
        
    val_examples = []
    print('val range:', train_num+1, train_num+val_num)
    for i in range(train_num+1, train_num+val_num):
        sentence1 = openfile.iloc[i]['s1']
        sentence2 = openfile.iloc[i]['s2']
        score = openfile.iloc[i]['cosSim']
        val_examples .append({i:[sentence1, sentence2, np.float32(score)]})
        
    test_examples = []
    print('test range:', train_num+val_num+1, total)
    for i in range(train_num+val_num+1, total):
        sentence1 = openfile.iloc[i]['s1']
        sentence2 = openfile.iloc[i]['s2']
        score = openfile.iloc[i]['cosSim']
        test_examples.append({i:[sentence1, sentence2, np.float32(score)]})

    return train_examples, val_examples, test_examples


In [5]:
train_examples, val_examples, test_examples = make_sts_input_example(staDF)


train range: 10514
val range: 10515 12485
test range: 12486 13142
